In [0]:
import pandas as pd
from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn import metrics
from sklearn.datasets import make_blobs
import numpy as np

In [5]:
from google.colab import files
datafile = files.upload()

Saving data.csv to data.csv


In [0]:
data = pd.read_csv('data.csv')

In [0]:
sample = data.sample(frac=0.1)
y = sample['category_name']

In [0]:
def try_vectorizer(v, cluster_alg):
  X = v.fit_transform(sample['title'])
  cluster = cluster_alg
  cluster.fit(X)
  labels = cluster.labels_
  #print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels))
  #print("Completeness: %0.3f" % metrics.completeness_score(y, labels))
  #print("V-measure: %0.3f" % metrics.v_measure_score(y, labels))
  return metrics.v_measure_score(y, labels)

In [0]:
cluster = KMeans(n_clusters=47, random_state=42)

In [0]:
min_df_values = np.arange(0.001, 0.01, 0.001)

In [0]:
max_df_values = np.arange(0.01, 0.1, 0.01) #при max_df>0.1 ничего не меняется

In [0]:
ngram_range_values = [(1,1), (1,2), (2,2), (1,3), (2,3), (3,3)]

In [0]:
def compare_min_df (cluster_alg, param_values):
  cv, tfidf = CountVectorizer, TfidfVectorizer
  cv_score, tfidf_score = 0, 0
  for value in param_values:
    measure_cv = try_vectorizer(cv(min_df=value), cluster_alg)
    measure_tfidf = try_vectorizer(tfidf(min_df=value), cluster_alg)
    print('min_df={:.4f}, cv:{:.4f}, tfidf: {:.4f}'.format( value,  measure_cv, measure_tfidf))
    if measure_cv > measure_tfidf:
      cv_score += 1 
    if measure_tfidf > measure_cv:
      tfidf_score += 1
  print('cv_score:{}, tfidf_score:{}'.format(cv_score,  tfidf_score))

In [0]:
def compare_max_df (cluster_alg, param_values):
  cv, tfidf = CountVectorizer, TfidfVectorizer
  cv_score, tfidf_score = 0, 0
  for value in param_values:
    measure_cv = try_vectorizer(cv(max_df=value), cluster_alg)
    measure_tfidf = try_vectorizer(tfidf(max_df=value), cluster_alg)
    print('max_df={:.3f}, cv: {:.4f}, tfidf: {:.4f}'.format( value,  measure_cv, measure_tfidf))
    if measure_cv > measure_tfidf:
      cv_score += 1 
    if measure_tfidf > measure_cv:
      tfidf_score += 1
  print('cv_score:{}, tfidf_score:{}'.format(cv_score,  tfidf_score))

In [0]:
def compare_ngram_range (cluster_alg, param_values):
  cv, tfidf = CountVectorizer, TfidfVectorizer
  cv_score, tfidf_score = 0, 0
  for value in param_values:
    measure_cv = try_vectorizer(cv(ngram_range=value), cluster_alg)
    measure_tfidf = try_vectorizer(tfidf(ngram_range=value), cluster_alg)
    print('ngram_range={}, cv: {:.4f}, tfidf: {:.4f}'.format( value,  measure_cv, measure_tfidf))
    if measure_cv > measure_tfidf:
      cv_score += 1 
    if measure_tfidf > measure_cv:
      tfidf_score += 1
  print('cv_score:{}, tfidf_score:{}'.format(cv_score,  tfidf_score))

In [0]:
def compare (cluster_alg):
  compare_min_df(cluster_alg, min_df_values)
  compare_max_df(cluster_alg, max_df_values)
  compare_ngram_range(cluster_alg, ngram_range_values)

##K-means

In [25]:
compare(cluster)

min_df=0.001, cv:0.3174, tfidf: 0.3182
min_df=0.002, cv:0.3237, tfidf: 0.3288
min_df=0.003, cv:0.3260, tfidf: 0.3328
min_df=0.004, cv:0.3187, tfidf: 0.3312
min_df=0.005, cv:0.3198, tfidf: 0.3311
min_df=0.006, cv:0.3107, tfidf: 0.3132
min_df=0.007, cv:0.3046, tfidf: 0.3117
min_df=0.008, cv:0.3033, tfidf: 0.3105
min_df=0.009, cv:0.2957, tfidf: 0.3025
cv_score:0, tfidf_score:9
max_df=0.010, cv: 0.1906, tfidf: 0.1773
max_df=0.020, cv: 0.2446, tfidf: 0.2732
max_df=0.030, cv: 0.2446, tfidf: 0.2732
max_df=0.040, cv: 0.2674, tfidf: 0.2872
max_df=0.050, cv: 0.2620, tfidf: 0.2870
max_df=0.060, cv: 0.3051, tfidf: 0.3385
max_df=0.070, cv: 0.3120, tfidf: 0.3370
max_df=0.080, cv: 0.3120, tfidf: 0.3370
max_df=0.090, cv: 0.3222, tfidf: 0.3196
cv_score:2, tfidf_score:7
ngram_range=(1, 1), cv: 0.3222, tfidf: 0.3196
ngram_range=(1, 2), cv: 0.2411, tfidf: 0.3006
ngram_range=(2, 2), cv: 0.1105, tfidf: 0.1087
ngram_range=(1, 3), cv: 0.2183, tfidf: 0.2931
ngram_range=(2, 3), cv: 0.0983, tfidf: 0.1045
ngram_r

При разных параметрах max_df и min_df векторизация tfidf показывает лучший результат. При изменении параметра ngram_range всё не так однозначно. В целом, для кластеризации K-Means tfidf кажется предпочтительнее

##Mini Batch K-means

In [0]:
sample = data.sample(frac=1)
y = sample['category_name']

In [0]:
cluster_MKB = MiniBatchKMeans(n_clusters=1000, init_size=5000, max_iter=5000, 
                          max_no_improvement=100, reassignment_ratio=0.3)


In [0]:
compare(cluster_MKB)

min_df=0.001, cv:0.3932, tfidf: 0.3927
min_df=0.002, cv:0.3692, tfidf: 0.3651
min_df=0.003, cv:0.3549, tfidf: 0.3549
min_df=0.004, cv:0.3406, tfidf: 0.3400
min_df=0.005, cv:0.3200, tfidf: 0.3190
min_df=0.006, cv:0.3055, tfidf: 0.3045
min_df=0.007, cv:0.3058, tfidf: 0.3052
min_df=0.008, cv:0.3030, tfidf: 0.3027
min_df=0.009, cv:0.3005, tfidf: 0.3008
cv_score:8, tfidf_score:1
max_df=0.010, cv: 0.3842, tfidf: 0.3517
max_df=0.020, cv: 0.3987, tfidf: 0.3763
max_df=0.030, cv: 0.3988, tfidf: 0.4064
max_df=0.040, cv: 0.4024, tfidf: 0.4195
max_df=0.050, cv: 0.3927, tfidf: 0.4085
max_df=0.060, cv: 0.4136, tfidf: 0.4226
max_df=0.070, cv: 0.4061, tfidf: 0.4210
max_df=0.080, cv: 0.4111, tfidf: 0.4199
max_df=0.090, cv: 0.4066, tfidf: 0.4223
cv_score:2, tfidf_score:7
ngram_range=(1, 1), cv: 0.4131, tfidf: 0.4241
ngram_range=(1, 2), cv: 0.3955, tfidf: 0.4060
ngram_range=(2, 2), cv: 0.3033, tfidf: 0.2487


In [0]:
sample = data.sample(frac=0.1) #всё скрашилось, поэтому уменьшим выборку и сравним разные ngram_range заново
y = sample['category_name']

In [20]:
compare_ngram_range (cluster_MKB, ngram_range_values)

ngram_range=(1, 1), cv: 0.4229, tfidf: 0.4349
ngram_range=(1, 2), cv: 0.4002, tfidf: 0.4080
ngram_range=(2, 2), cv: 0.2522, tfidf: 0.2795
ngram_range=(1, 3), cv: 0.3862, tfidf: 0.3865
ngram_range=(2, 3), cv: 0.2329, tfidf: 0.2245
ngram_range=(3, 3), cv: 0.2250, tfidf: 0.2139
cv_score:2, tfidf_score:4


Здесь был другой тип кластеризации и размер выборки. При изменении параметра min_df чаще оказывался лучше Countvectorizer, однако разница почти всегда незначительная(меньше 1%). При изменении max_df лучше оказывался tfidf с более значительным перевесом (7 из 9 раз). При 6 разных параметрах ngram_range в 4 случаях лучше оказалось tfidf. Результаты менее однозначные, чем для K-means, но можно сказать, что tfidf всё же лучше. 


##Affinity Propagation

In [0]:
sample = data.sample(frac=0.005)
y = sample['category_name']

In [0]:
cluster_AP = AffinityPropagation(damping=0.7, preference=-2, 
                              max_iter=400, convergence_iter=10)

In [0]:
min_df_values = np.arange(0.0005, 0.003, 0.0005) # поменяем значения min_df, иначе возникает ConvergenceWarning

In [31]:
compare(cluster_AP)

min_df=0.0005, cv:0.6045, tfidf: 0.5412
min_df=0.0010, cv:0.4960, tfidf: 0.5614
min_df=0.0015, cv:0.4960, tfidf: 0.5614
min_df=0.0020, cv:0.4776, tfidf: 0.4758
min_df=0.0025, cv:0.4719, tfidf: 0.4753
cv_score:2, tfidf_score:3
max_df=0.010, cv: 0.5743, tfidf: 0.2568
max_df=0.020, cv: 0.5911, tfidf: 0.2985
max_df=0.030, cv: 0.5946, tfidf: 0.2909
max_df=0.040, cv: 0.5954, tfidf: 0.3096
max_df=0.050, cv: 0.6030, tfidf: 0.5369
max_df=0.060, cv: 0.6033, tfidf: 0.5372
max_df=0.070, cv: 0.6033, tfidf: 0.5372
max_df=0.080, cv: 0.6045, tfidf: 0.5368
max_df=0.090, cv: 0.6045, tfidf: 0.5412
cv_score:9, tfidf_score:0
ngram_range=(1, 1), cv: 0.6045, tfidf: 0.5412
ngram_range=(1, 2), cv: 0.5962, tfidf: 0.5377
ngram_range=(2, 2), cv: 0.5388, tfidf: 0.3867
ngram_range=(1, 3), cv: 0.5959, tfidf: 0.5345
ngram_range=(2, 3), cv: 0.5692, tfidf: 0.3845
ngram_range=(3, 3), cv: 0.5689, tfidf: 0.4832
cv_score:6, tfidf_score:0


CountVectorizer выглядит определенно лучше для данного типа кластеризации

На нескольких алгоритмах кластеризации проверьте, какое матричное разложение (TruncatedSVD или NMF) работает лучше для кластеризации. (3 балла)

In [0]:
def try_SVD(cluster, v):
  X_cv = v.fit_transform(sample['title'])
  svd = TruncatedSVD(50, random_state=42)
  X_svd = svd.fit_transform(X_cv)
  cluster.fit(X_svd)
  labels = cluster.labels_
  print('SVD: ', metrics.v_measure_score(y, labels))

In [0]:
def try_NMF(cluster, v):
  X_cv = v.fit_transform(sample['title'])
  nmf = NMF(50, random_state=42)
  X_nmf = nmf.fit_transform(X_cv)
  cluster.fit(X_nmf)
  labels = cluster.labels_
  print('NMF: ', metrics.v_measure_score(y, labels))

In [0]:
def SVD_or_NMF(cluster, v):
  try_SVD (cluster, v)
  try_NMF(cluster, v)


## AgglomerativeClustering

In [0]:
cv = CountVectorizer(min_df=2, max_df=0.4)
tfidf = TfidfVectorizer(min_df=2, max_df=0.4)

In [0]:
sample = data.sample(frac=0.05)
y = sample['category_name']

In [0]:
cluster_A = AgglomerativeClustering(n_clusters=47)

In [78]:
SVD_or_NMF(cluster_A, cv)

SVD:  0.3392612049780693
NMF:  0.3294212382847426


In [79]:
SVD_or_NMF(cluster_A, tfidf)

SVD:  0.35379590715189563
NMF:  0.3507134040964364


##DBSCAN

In [0]:
cluster_DB = DBSCAN(min_samples=10, eps=0.3) 

In [80]:
SVD_or_NMF(cluster_DB, cv)

SVD:  0.31572876997343213
NMF:  0.008809883748288422


In [81]:
SVD_or_NMF(cluster_DB, tfidf)

SVD:  0.15860834128120185
NMF:  -3.760773223866692e-16


##Mean Shift

In [0]:
cluster_MS = MeanShift(cluster_all=False, bandwidth=0.5)

In [82]:
SVD_or_NMF(cluster_MS, cv)

SVD:  0.34766448142697637
NMF:  0.019167016079662716


In [83]:
SVD_or_NMF(cluster_MS, tfidf)

SVD:  0.30921393963454474
NMF:  -3.760773223866692e-16


Во всех случаях TruncatedSVD разложение оказалось лучше. 

С помощью алгоритмов, умеющих выделять выбросы, попробуйте найти необычные объявления (необычные - это такие, которые непонятно к какой категории можно вообще отнести, что-то с ошибками или вообще какая-то дичь). В этом задании можно использовать любую векторизацию. (4 балла)

In [0]:
def labels(cluster_alg):
  cv = CountVectorizer(min_df = 0.001, max_df=0.4)
  X = cv.fit_transform(sample['title'])
  svd = TruncatedSVD(50)
  cluster = cluster_alg
  X_svd = svd.fit_transform(X)
  cluster.fit(X_svd)
  labels = cluster.labels_
  #sample['cluster'] = cluster.labels_
  #print(sample[sample.cluster==-1].head(20))
  return labels

In [0]:
sample = data.sample(frac=0.05)
y = sample['category_name']

In [0]:
sample['cluster'] = labels(cluster_DB)

In [123]:
sample[sample.cluster==-1].head(20)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,cluster
90621,917f3e3fc140,c21ca6bba4e9,Татарстан,Казань,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,29,Ботинки и сапоги,"Продаю ботинки унисекс Котофей 29 размера,желт...",500.0,70,2017-04-14,Private,bb011a178f0c167b6837a329b558b994f5c869e913b188...,88.0,-1
97670,86b7a0b8a67f,740c028b7e54,Самарская область,Тольятти,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,29,Сапоги+ботинки,Отдам за шоколадку сапоги+ботинки на девочку 29р,50.0,74,2017-04-13,Private,5dba21139839bd707b9b8f081b382e5c3d67c5760770b3...,91.0,-1
147018,fa54519cdb85,98afa66df686,Новосибирская область,Новосибирск,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Продам детское кресло 0-18 кг,"Продам детское кресло 0-18 кг., регулировка на...",1500.0,50,2017-04-13,Private,28b3d4168401d025932bcd1d57fa644a332409073613b9...,796.0,-1
87072,90fde7a6c85e,4be8b14936aa,Свердловская область,Нижний Тагил,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),Юбка новая стрейч,"Ликвидация остатков, Турция, длина 49 см",350.0,67,2017-04-18,Private,466eb83bcaffc321dada0e62fe9e07a68d45970b62a083...,375.0,-1
191296,a29f215a41a2,57364177157d,Оренбургская область,Оренбург,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,20,Ботинки для мальчика,Ботинки капика в отличном состоянии. Размер 20,800.0,45,2017-04-13,Private,078cd0fa476c470f7936ecda1a7bc94586531eb912db98...,46.0,-1
219102,0be33b7d83f4,7f417856264d,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска 3 в 1 (новая),"Коляска ""Tutis Zippy"", 3 в 1/\n/\nЦвет: коричн...",21500.0,2,2017-04-14,Private,123cd512bf36ee3e966a9258ad7f2e138de614f11958b2...,1002.0,-1
124782,dec0065dd0a7,9b959398b331,Иркутская область,Усть-Кут,Недвижимость,Гаражи и машиноместа,Продам,Гараж,Кирпичный,Гараж,Гараж в районе ЯГУ. Все вопросы по телефону,300000.0,68,2017-04-12,Private,NaN,NaN,-1
211590,043e65f81bf2,ac157d80cf56,Пермский край,Уральский,Недвижимость,"Дома, дачи, коттеджи",Продам,Дача,NaN,Дача 20 м² на участке 6 сот.,Продается садовый участок. Ухоженный с плодовы...,210000.0,15,2017-04-13,Private,ed27c7643480f02b84ddd8fede278cc1b57126687e6494...,2218.0,-1
36015,289db1c5dcb3,dae07871c6df,Башкортостан,Уфа,Недвижимость,Коммерческая недвижимость,Сниму,Складское помещение,NaN,Сниму складское помещение,Сниму теплое складское помещение 30-50 кв.м в...,NaN,7,2017-04-17,Private,NaN,NaN,-1
97647,e122288e9ca2,c2ac8e2c3fc0,Саратовская область,Саратов,Хобби и отдых,Велосипеды,Детские,NaN,NaN,Детский велосипед Safari BMX-bike 18,"Спускает колесо, состояние на 4, передний торм...",3000.0,10,2017-04-12,Private,40c522d39671fe85e9710f9d9052f3ee6b2a8163a9a555...,2370.0,-1


"Жёсткий на 40gb ide samsung" - пропущено слово "диск", прилагательное может вполне встретиться и в описании матраса, "40gb ide samsung" - можно отнести к телефону, аксессуару или гаджету, например . "Сумка для ноута новая" - можно понять как аксессуар, относящийся к категории "одежда"(подраздел аксессуары), а не "электроника", слово "ноут" не так частотно. "Сумка натуральная замша и кожа" - возможно это сумка для гаджета и должна тогда относиться к "электронике" и аксессуарам для электроники, если предыдущий пример считать верным. 

In [135]:
sample['cluster'] = labels(cluster_MS)
len(sample[sample.cluster==-1])

216

In [136]:
sample[sample.cluster==-1].head(20)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,cluster
160627,7c31978d51e1,96555f33b381,Алтайский край,Хабары,Транспорт,Мотоциклы и мототехника,Мопеды и скутеры,NaN,NaN,Мопед детский,Детский мопед,8000.0,31,2017-04-18,Private,ae540421442444a0b69a9c6b6bc26addeff6d56d874abd...,1136.0,-1
49563,5c95f16d3692,dd3376308e14,Волгоградская область,Палласовка,Бытовая электроника,Ноутбуки,NaN,NaN,NaN,Ноутбук Acer E1-531,Продам ноутбук Acer E1-531 в хорошем состоянии...,8500.0,90,2017-04-18,Company,579f5574eb4a34949d52bd4e9e2bab243f52e703d20c31...,2963.0,-1
66327,a052b7fb1e78,8218d5b04283,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,20,Босоножки детские котофей,Мягкие босоножки котофей. Размер 19. Состояни...,1000.0,38,2017-04-12,Private,97ac76775ac783713e7bb2d3a5ebada6cc8100c96e8db1...,93.0,-1
65806,5d41a73d595a,f3241e214d2c,Краснодарский край,Краснодар,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Босоножки,Продаются новые босоножки. Натуральная кожа. Р...,1000.0,18,2017-04-18,Private,76ee5c5a23b6b422f29494cea9a5d5e8e99fa73e997b18...,44.0,-1
67343,f1fdde7f2eeb,29559ffd63b7,Башкортостан,Сибай,Бытовая электроника,Ноутбуки,NaN,NaN,NaN,Ноутбук,"Продам ноутбук Toshiba satellite L300D-14 N, н...",1000.0,13,2017-04-16,Private,91cb661e65cb5795c3d842f3675117d93cd685b4fc2b7c...,2959.0,-1
221639,bccccae809d0,1d9d6a3c0072,Калининградская область,Калининград,Личные вещи,Детская одежда и обувь,Для мальчиков,Верхняя одежда,122-128 см (6-8 лет),Комплект зимний,Продам зимний комплект в хорошем состоянии . ...,1000.0,57,2017-04-14,Private,9c0e7613271242bde2c3e89fa404616ee84bbaaa6090a6...,50.0,-1
150863,f441cfb17e07,168634934158,Свердловская область,Первоуральск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,38,Босоножки на танкетке,"Продам замшевые босоножки на танкетке, немного...",1500.0,10,2017-04-12,Private,56dcdbddb2ab02ee5e97a9027aeafd1ddc3adcf0fc6fa0...,440.0,-1
242498,e477fd816004,1338c6e6aae6,Самарская область,Самара,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,"Ванна детская овальная ""Фаворит"" 100 см (tm Du...","Ванна детская овальная ""Фаворит"" 100 см (tm Du...",550.0,1960,2017-04-18,Company,ce5b3166eb3d04e30a7bce91a2ae7a426f6c76613786cf...,1019.0,-1
92149,8243dbb6f96f,6a0d9945528a,Ярославская область,Ярославль,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,38,Лакированные босоножки,Предлагаю красивые лакированные кожаные босоно...,1200.0,6,2017-04-13,Private,c587b9104265030acb45d335111d32c2e2df2b81c5660c...,504.0,-1
42284,274d20a49314,41803bc0b458,Башкортостан,Уфа,Бытовая электроника,Ноутбуки,NaN,NaN,NaN,Asus. Мощный. Комплект. i3. 8 гб. GrForce 2 гб,"Продаю ноутбук Asus. Модель F552CL. Коробка, з...",12000.0,6,2017-04-14,Private,ef039db7c87839de978b0a7afebe3eb1c13694e09d9484...,2951.0,-1


"Мопед детский". Скорее всего прилагательное "детский" встречается чаще в категории "детская одежда" и не так часто в категории "Мотоциклы и мототехника". "Комплект зимний" - слишком общее описание в категории "детская одежда", слово "комплект" вполне может встречаться и в бытовой технике. "Ванна детская овальная "Фаворит"", видимо, слово "ванна" частотнее в заголовках категории "для дома и для дачи", а не "Товары для детей и игрушки". Часто встречаются "босоножки" как в детских товарах, так и в женской одежде, что тоже создают неоднозначность. "Все по 50" - слишком общее описание, могут быть книги, а не стройматериалы. "Детский комод" - возможно слово "детский" снова сбивает с толку